Deal with unlabeled bbox (remove car IDs)
Goal: associate measurements to unique objects
1. try stitching objects in mutl-camera road

In [44]:
from utils import *
import importlib
import utils
importlib.reload(utils)
import os.path
from os import path
import pandas as pd
import utils_optimization as opt
importlib.reload(opt)
import data_association as da
importlib.reload(da)
import time
from scipy.optimize import minimize,NonlinearConstraint,leastsq,fmin_slsqp,least_squares
import numpy.linalg as LA
from shapely.geometry import Polygon

In [45]:
# read & rectify each camera df individually

data_path = pathlib.Path().absolute().joinpath('../June_5min')
tform_path = pathlib.Path().absolute().joinpath('../tform')

# df = pd.DataFrame()
for root,dirs,files in os.walk(str(data_path), topdown = True):
    for file in files:
        if file.endswith(".csv") and file.startswith('p1c2_1'):
            
            # ... reading & preprocessing ...
            print(file)
            file_path = data_path.joinpath(file)
            df = utils.preprocess(file_path, tform_path, skip_row = 0)
            df = df.sort_values(by=['Frame #','Timestamp']).reset_index(drop=True)
            df = utils.preprocess_data_association(df)
            
            # ... rectifying ...
            df = opt.rectify(df)
            
            # ... post processing ...
            df = utils.post_process(df)
            
            # ... saving ...
            new_name = 'rectified_' + file
            df.to_csv(data_path.joinpath('rectified/',new_name), index=False)
    break

p1c2_1_track_outputs_3D.csv
Reading data...


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2910: DtypeWarning: Columns (0,1,2,4,5,6,7,8,9,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,40,41,42,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Total # cars before preprocessing: 1377
Transform from image to road for p1c2...
Interpret missing timestamps...
No timestamps values
Constrain x,y range by camera FOV
100 220 -5 45
Filtering out tailing place holders...
Get the longest continuous frame chuck...
Get x direction...
Naive filter...
after direction=0 filter:  996


C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:67: RuntimeWarning: invalid value encountered in greater
  outliers =	np.logical_or(abs(w1 - m) > 2 * s,abs(w2 - m) > 2 * s)
C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:77: RuntimeWarning: invalid value encountered in greater
  outliers =	np.logical_or(abs(l1 - m) > 2 * s,abs(l2 - m) > 2 * s)


filter width length: 996
Before DA:  996 cars
After stitching:  893 cars, checking for overlaps...


KeyboardInterrupt: 

In [37]:
import utils
importlib.reload(utils)
import data_association as da
importlib.reload(da)

parent = da.stitch_objects(df)

In [38]:
parent

{121: 121,
 1121: 1121,
 2121: 2121,
 3121: 2121,
 4121: 4121,
 5121: 5121,
 6121: 6121,
 7121: 7121,
 8121: 8121,
 9121: 9121,
 10121: 2121,
 11121: 11121,
 13121: 13121,
 18121: 18121,
 21121: 21121,
 23121: 23121,
 24121: 24121,
 26121: 26121,
 30121: 30121,
 32121: 32121,
 33121: 33121,
 40121: 40121,
 47121: 47121,
 53121: 53121,
 56121: 56121,
 60121: 60121,
 64121: 64121,
 77121: 77121,
 84121: 84121,
 85121: 85121,
 96121: 96121,
 97121: 97121,
 107121: 107121,
 116121: 116121,
 121121: 121121,
 133121: 133121,
 137121: 137121,
 140121: 140121,
 143121: 143121,
 145121: 145121,
 148121: 148121,
 149121: 149121,
 150121: 150121,
 161121: 161121,
 180121: 180121,
 181121: 181121,
 182121: 182121,
 187121: 187121,
 194121: 194121,
 195121: 195121,
 208121: 208121,
 219121: 219121,
 228121: 228121,
 235121: 182121,
 237121: 237121,
 250121: 181121,
 253121: 253121,
 256121: 256121,
 261121: 261121,
 263121: 263121,
 267121: 267121,
 279121: 279121,
 283121: 283121,
 288121: 288121,

In [39]:
gl = parent.keys()
parent = da.compress(parent,gl)

In [40]:
parent

{121: 121,
 1121: 1121,
 2121: 2121,
 3121: 2121,
 4121: 4121,
 5121: 5121,
 6121: 6121,
 7121: 7121,
 8121: 8121,
 9121: 9121,
 10121: 2121,
 11121: 11121,
 13121: 13121,
 18121: 18121,
 21121: 21121,
 23121: 23121,
 24121: 24121,
 26121: 26121,
 30121: 30121,
 32121: 32121,
 33121: 33121,
 40121: 40121,
 47121: 47121,
 53121: 53121,
 56121: 56121,
 60121: 60121,
 64121: 64121,
 77121: 77121,
 84121: 84121,
 85121: 85121,
 96121: 96121,
 97121: 97121,
 107121: 107121,
 116121: 116121,
 121121: 121121,
 133121: 133121,
 137121: 137121,
 140121: 140121,
 143121: 143121,
 145121: 145121,
 148121: 148121,
 149121: 149121,
 150121: 150121,
 161121: 161121,
 180121: 180121,
 181121: 181121,
 182121: 182121,
 187121: 187121,
 194121: 194121,
 195121: 195121,
 208121: 208121,
 219121: 219121,
 228121: 228121,
 235121: 182121,
 237121: 237121,
 250121: 181121,
 253121: 253121,
 256121: 256121,
 261121: 261121,
 263121: 263121,
 267121: 267121,
 279121: 279121,
 283121: 283121,
 288121: 288121,

In [26]:
import utils
importlib.reload(utils)
import data_association as da
importlib.reload(da)
df = utils.preprocess_data_association(df)

Before DA:  996 cars
3121 10121
235121 182121
328121 331121
360121 328121
328121 331121
398121 390121
476121 443121
479121 453121
636121 648121
784121 747121
840121 834121
859121 867121
882121 901121
898121 881121
922121 920121
925121 932121
932121 930121
930121 913121
928121 925121
925121 913121
930121 913121
932121 913121
944121 920121
1004121 954121
1048121 1051121
1199121 1135121
1282121 1274121
1287121 1285121
1294121 1224121
1406121 1353121
1462121 1463121
1465121 1462121
1462121 1463121
1594121 1569121
1613121 398121
398121 390121
1627121 1613121
1613121 390121
1669121 1634121
1769121 1735121
1794121 1795121
1799121 1783121
1863121 1816121
2068121 2014121
2070121 2014121
2079121 925121
925121 913121
2106121 2079121
2079121 913121
2108121 932121
932121 913121
2109121 2083121
2110121 2079121
2079121 913121
2128121 2111121
2152121 1566121
2167121 1316121
2170121 2152121
2152121 1566121
2419121 2380121
2430121 2388121
2432121 2414121
2528121 925121
925121 913121
2560121 2528121
2528

2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 

2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 

2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 2568121
2568121 2571121
2571121 2575121
2575121 

RecursionError: maximum recursion depth exceeded in comparison

In [26]:
# read and preprocess data (one csv file)

data_path = pathlib.Path().absolute().joinpath('../June_5min')
tform_path = pathlib.Path().absolute().joinpath('../tform')
file_path = data_path.joinpath('p1c1_1_track_outputs_3D.csv')

df = utils.preprocess(file_path, tform_path, skip_row = 0)

Reading data...
Total # cars before preprocessing: 1810
Transform from image to road for p1c1...
Interpret missing timestamps...
No timestamps values
Constrain x,y range by camera FOV
0 130 -5 45
Filtering out tailing place holders...
Get the longest continuous frame chuck...
Get x direction...
Naive filter...
after direction=0 filter:  1115


C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:67: RuntimeWarning: invalid value encountered in greater
  outliers =	np.logical_or(abs(w1 - m) > 2 * s,abs(w2 - m) > 2 * s)
C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:77: RuntimeWarning: invalid value encountered in greater
  outliers =	np.logical_or(abs(l1 - m) > 2 * s,abs(l2 - m) > 2 * s)


filter width length: 1115


In [32]:
# stitching based on prediction and overlaps

import utils
importlib.reload(utils)

df = utils.preprocess_data_association(df)


Before DA:  225 cars
Frame :    0
Frame :  100
Frame :  200
Frame :  300
Frame :  400
Frame :  500
Frame :  600
Frame :  700
Frame :  800
Frame :  900
Frame : 1000
Frame : 1100
Frame : 1200
Frame : 1300
Frame : 1400
Frame : 1500
Frame : 1600
Frame : 1700


C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:775: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'] = df['ID'].apply(lambda x: parent[x] if x in parent else x)


After stitching:  210 cars


C:\Users\wangy79\Documents\I24_trajectory\I24-trajectory-generation\utils.py:780: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ID'] = df['ID'].apply(lambda x: parent[x] if x in parent else x)


After assocating overlaps:  201 cars
Get the longest continuous frame chunk...


100%|██████████████████████████████████████████████████████████████████████████████| 1798/1798 [00:26<00:00, 68.61it/s]


In [33]:
# try rectification
import utils_optimization as opt
importlib.reload(opt)
df = opt.rectify(df) # parallel, batch method

100%|████████████████████████████████████████████████████████████████████████████████| 201/201 [05:20<00:00,  1.59s/it]


In [34]:
df = utils.post_process(df)

cap width at 2.59m...
standardize format for plotter...


In [35]:
df.to_csv('../June_5min/rectified/rectified_p1c1_uncorrected_5min_1min.csv',index=False)

In [68]:
# make an animation based on LMCS
import os
import glob
import importlib
import animation_utils as an
importlib.reload(an)
image_folder = '../FramePic'

filelist = glob.glob(os.path.join(image_folder, "*"))
for f in filelist:
    os.remove(f)
    
if len(df['camera'].unique())==1:
    dim0 = get_camera_range(df['camera'][0])
else:
    dim0 = get_camera_range('all')
print(dim0)
dim = [d * 3.281 for d in dim0] # convert meter to feet
an.generate_frames(df, dim, skip_frame=1, image_folder=image_folder)

(0, 130, -5, 45)
Frame:  0 8994


In [69]:
# Fetch image files from the folder, and create an animation.
importlib.reload(an)
video_name = '../p1c1_uncorrected5min_DA.mp4'
an.write_video(image_folder, video_name, fps=30)

In [14]:
# visualize footprint on the camera video

import plot_rectified_objects
importlib.reload(plot_rectified_objects)

video = str(data_path.joinpath('raw_video/p1c1_1.mp4'))
label_file = str(data_path.joinpath('rectified/rectified_p1c1_1_track_outputs_3D2.csv')) 
# # label_file = str(data_path.joinpath('rectified/p1_all/rectified_p1c3.csv')) 
# # label_file = str(data_path.joinpath('rectified/uncorrected_individual/rectified_p1c3_uncorrected.csv'))
# # label_file = str(data_path.joinpath('rectified/uncorrected_all/rectified_p1c3.csv'))
plot_rectified_objects.plot_vehicle_csv(video,label_file, frame_rate = 10,show_2d = False,show_3d = True,show_LMCS = True,show_rectified = False, ds=True)


In [78]:
label_file

'C:\\Users\\wangy79\\Documents\\I24_trajectory\\I24-trajectory-generation\\..\\June_5min\\rectified\\rectified_p1c1_uncorrected_5min.csv'

In [ ]:
# read & rectify each camera df individually
import utils
importlib.reload(utils)
data_path = pathlib.Path().absolute().joinpath('../June 2021 Data - 1 minute 5 cameras w RAV 4/')
tform_path = pathlib.Path().absolute().joinpath('../2D-3D May 2021/i24-roadway-transforms-main/tform')

# df = pd.DataFrame()
for root,dirs,files in os.walk(str(data_path), topdown = True):
    for file in files:
        if file.endswith(".csv") and file.startswith('p1c6'):
            
            # ... reading ...
            file_name = data_path.joinpath(file)
            camera_id = utils.find_camera_name(file)
            print('reading ',camera_id,'...')
            
            # ... preprocessing ...
            df = utils.preprocess(file_name, tform_path)
            df = df.assign(camera=camera_id)
            df = df.sort_values(by=['Frame #','Timestamp']).reset_index(drop=True)
        
            # ... rectifying ...
            df = opt.rectify(df)
            
            # ... post processing ...
            dfpp = utils.post_process(df)
            dfpp_img = utils.road_to_img(dfpp, tform_path)
            
            # ... saving ...
            new_name = 'rectified_' + camera_id +'.csv'
            dfpp_img.to_csv(new_name)
    break

In [ ]:
# plot 
import utils
importlib.reload(utils)
D = np.array(df[['bbr_x','bbr_y', 'fbr_x','fbr_y','fbl_x','fbl_y','bbl_x', 'bbl_y']])
utils.plot_track(D,15,5)

In [ ]:
fig, ax = plt.subplots()
# plt.xlim(xmin, xmax)
# plt.ylim(20, 42) # 0, 42
plt.xlabel('meter')
plt.ylabel('meter')
#plot design setup
plt.axhline(y = 21.25, color = 'black', linestyle = 'solid')
plt.axhline(y = 24, color = 'yellow', linestyle = '--')
plt.axhline(y = 18.5, color = 'yellow', linestyle = '--')
carColors=cm.rainbow(np.linspace(0,1,5))

# frameSnap = np.array(group[['bbr_x','bbr_y','fbr_x','fbr_y','fbl_x','fbl_y','bbl_x', 'bbl_y']])
for carid, track in tracks.items():
    frameSnap = track
    c = carColors[carid%5]
    for j in range(len(frameSnap)):  # draw one vehicle onto the plot
    #     carID = int(frameSnap[j,0])  # Finding the unique carID for each vehicle in the frame
        coord = frameSnap[j,0:8]   # Finding the coordinates of the car
        if (np.isnan(coord).any()):
            print('nan')
            continue
        coord = np.reshape(coord,(-1,2)).tolist()
        coord.append(coord[0])
        xs, ys = zip(*coord)
        plt.plot(xs,ys,c=c,label=carid if j==0 else '')
        #     plt.text(frameSnap[j,1], frameSnap[j,2], carID)
#     plt.legend()
